# Lecture 14: Hierarchial Models

## Instructor: 胡传鹏（博士）[Dr. Hu Chuan-Peng]

### 南京师范大学心理学院[School of Psychology, Nanjing Normal University]

### 回顾：广义线性模型GLM

**当因变量为离散变量**

比如，因变量为二分变量 (答题正确率)，其中1代表回答正确，0代表回答错误。

正确率为离散变量，服从伯努利(Bernoulli)分布。

![Image Name](https://cdn.kesci.com/upload/image/rloa62fn8w.png?imageView2/0/w/960/h/960)


对于因变量为离散变量的情况，我们需要使用广义线性模型(Generalized linear model，GLM)。

其特点为：
- 分布簇 (dist)不再局限于正态分布，而是允许其他不同的分布，比如 $y \sim Bernoulli(p)$。
- 需要 **链接函数$g()$** 将 $\alpha + \beta * x$  映射到 $p$所在的范围。

| 一般线性模型 | 广义线性模型 | 
|---|---|
| $y \sim Normal(\mu, \sigma)$ | $y \sim dist(p)$ |
| $\mu = \alpha + \beta *x$ | $p = g(z)$|
|  | $z = \alpha + \beta *x$|


链接函数的具体转化过程，以逻辑(logit)回归为例：
1. 令 $z = \alpha + \beta *x$，$\mu$的范围为 $(-\infty, +\infty)$。
2. $p = g(z)$，其中 $g()$ 为链接函数，输出结果 $p$ 的范围为 $(0,1)$。
3.  最后将 $p$ 输入到分布函数中 $y \sim Bernoulli(p)$。


![Image Name](https://cdn.kesci.com/upload/image/rloa6zyf5a.png?imageView2/0/w/600/h/600)

## Part 1: Hierarchical Models

本节的目的在与：了解如何使用贝叶斯分层模型 (Bayesian Hierarchical model)对分层结构的数据进行分析。

重点在于：
- 了解分层模型的基本概念：包括变化截距与变化斜率。
- 通过 PyMC3 分别拟合变化截距模型与变化斜率模型(包含变化截距)。


### 数据的层级结构
#### 我们想用工龄预测某个拥有4个学院的学校员工的工资
##### 收集到的数据如下：

![Image Name](https://cdn.kesci.com/upload/image/rmcutay7yg.png?imageView2/0/w/960/h/960)



![Image Name](https://cdn.kesci.com/upload/image/rmcuwtjhi2.png?imageView2/0/w/960/h/960)


如果需要使用工作年限预测工资水平，则最直觉的模型是线性模型：
$y \sim Normal(\mu, \sigma)$
$\mu  = \alpha + \beta *x_{years}$

由于该数据具有层级结构，个体的数据嵌套在不同的院系之下，而不同院系嵌在学校之下。

考虑到院长之间会有区别，而院系之内的个体之间可能同质性较高，因此，忽略院系可能会带来不准确的预测。

对于这种具有层级结构的数据，我们可能有不同的做法。


### 方式一：非池化参数(no pooling)

这种做法的假定是：各部门之间完全没有关联，不同部门之间彼此独立。

从统计上讲，是假定各组之间的参数没有关系，或者说是完全异质。

在这种情况下，该模型等价于每个部门各有一个广义线性模型。

即：

部门1：$y \sim Normal(\mu_1, \sigma)$
$\mu_1  = \alpha_{1} + \beta_{1} *x_{years}$

部门2：$y \sim Normal(\mu_1, \sigma)$
$\mu_1  = \alpha_2 + \beta_{2} *x_{years}$

...

部门j：$y \sim Normal(\mu_j, \sigma)$
$\mu  = \alpha_j + \beta_j *x_{years}$



![Image Name](https://cdn.kesci.com/upload/image/rmcwluwquv.png?imageView2/0/w/960/h/960)


### 方式二：完全池化参数(complete pooling)

这种做法假定各院系之间完全没有差异；

从数据分析角度而言，是认为各院系间共享同一个参数；即所有院系是完全同质的。

在这种情况下，该模型等价于每个院系共享一个广义线性模型。

即：

$y \sim Normal(\mu, \sigma)$
$\mu  = \alpha + \beta *x_{years}$



![Image Name](https://cdn.kesci.com/upload/image/rmcx1f4dlx.png?imageView2/0/w/960/h/960)


### 方式三：部分池化参数(partial pooling)

在这种方式中，我们假定各院系间有一定的关系关系，即他们是嵌套在同一个总体（学校）中。

部分池化参数就是各组的参数并非完全相同，但却是来自于某个特定分布。

分层模型（Hierarchcial model）即是对参数进行部分池化。

例如，对于正态分布模型来说，层级模型包括两部分：

Group level (组水平): 

$y_{i,j} \sim Normal(\mu_{j}, \sigma_{j})$

Population level  (总体水平):

$\mu_{j} \sim Normal(\mu, \sigma)$


![Image Name](https://cdn.kesci.com/upload/image/rmcyhbglbb.png?imageView2/0/w/960/h/960)


在线性模型中，模型参数通常会分布截距与斜率，因此，理论上，层级线性模型中的截距与斜率均可以分层进行估计。

在上述例子中：变化截距与变化斜率会随着不同部门变化，表达式如下：

![Image Name](https://cdn.kesci.com/upload/image/rmdh2ogd7u.png?imageView2/0/w/640/h/640)

在实际中，分层模型参数是否进行分层估计本身是灵活的，例如，在上述例子中，可能会有三种情况：

- 不同部门员工工资存在不同。
	- 这被称为变化截距，截距的变化代表不同部门间员工平均工资的差异。
	- ![Image Name](https://cdn.kesci.com/upload/image/rmanj9sw62.png?imageView2/0/w/640/h/640)
- 不同学校中，员工工作年限对工资的预测作用会发生变化。
	- 这被称为变化斜率，斜率在不同部门间的变化代表，工作年限对工资的预测作用会在不同部门将发生变化。
	- ![Image Name](https://cdn.kesci.com/upload/image/rmank74ytm.png?imageView2/0/w/640/h/640)
- 变化截距 + 变化斜率
	- ![Image Name](https://cdn.kesci.com/upload/image/rmankk5jgh.png?imageView2/0/w/640/h/640)

图片来源：http://mfviz.com/hierarchical-models/

结合上节课的GLM和这节课的HLM，若因变量为离散变量，数据为嵌套结构时，我们可以讲两者进行结合，这就是GLMM.


![Image Name](https://cdn.kesci.com/upload/image/rmeb2c6lqc.png?imageView2/0/w/960/h/960)


## Part 2: 示例1——调查数据

In [1]:
#加载需要使用的库
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm

np.random.seed(123)  # 变化数种子，确保随后生成的变化数相同

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


**具体实例**

基于泰国初等教育数据库，我们想探究，学生是否接受学前教育对之后学生是否留级的影响？

数据来源：泰国初等教育的全国调查 (Raudenbush & Bhumirat，1992)。
- 数据中的每一行表示一名学生。
- 因变量 `REPEAT` 为二分类变量，表示学生在初等教育期间是否留级，1 = 留级，0 = 非留级。
- 自变量为学生是否受过学前教育 `PPED` (0 = 否，1 = 是)。

数据来源：https://github.com/MultiLevelAnalysis/Datasets-third-edition-Multilevel-book/blob/master/chapter%206/Thaieduc/thaieduc.sav

In [2]:
# 加载数据
data = pd.read_csv("/home/mw/input/data1317/thaieduc.csv")
data.PPED = data.PPED.map({'yes':1,'no':0})

In [3]:
data.head()

,SCHOOLID,SEX,PPED,REPEAT,MSESC
0,10103,girl,0,0,0.88
1,10103,girl,0,0,0.88
2,10103,girl,1,0,0.88
3,10103,girl,1,0,0.88
4,10103,girl,1,0,0.88


### Workflow

在回顾广义线性模型 (GLM)与介绍分层线性模型 (HLM)的基础知识后，我们回到泰国初等教育的实例，并通过PyMC完成的贝叶斯分层广义线性模型建模的全过程 (full workflow)。

![Image Name](https://cdn.kesci.com/upload/image/rkvikqg9q6.png?imageView2/0/w/650/h/650)

### (1) 提出研究问题

使用泰国初等教育数据探究的研究问题为：
- 学生是否接受学前教育对学生留级的影响？
- 在考虑不同学校的分层结构时，接受学前教育对学生是否留级的影响有哪些变化？
- 分层模型是否能提供额外的信息？

### (2) 数据收集

数据来源：泰国初等教育的全国调查 (Raudenbush & Bhumirat，1992)。
- 数据中的每一行表示一名学生。
- 因变量 `REPEAT` 为二分类变量，表示学生在初等教育期间是否留级，1 = 留级，0 = 非留级。
- 自变量为学生是否受过学前教育 `PPED` (0 = 否，1 = 是)。
- 其他变量包括学生性别 `SEX`、学生所在的学校 `SCHOOLID`和学校平均社会经济地位 (SES)分数 `MSESC`。

数据来源：https://github.com/MultiLevelAnalysis/Datasets-third-edition-Multilevel-book/blob/master/chapter%206/Thaieduc/thaieduc.sav

In [4]:
# 加载数据
data = pd.read_csv("/home/mw/input/data1317/thaieduc.csv")
data.PPED = data.PPED.map({'yes':1,'no':0})

In [5]:
data.head()

,SCHOOLID,SEX,PPED,REPEAT,MSESC
0,10103,girl,0,0,0.88
1,10103,girl,0,0,0.88
2,10103,girl,1,0,0.88
3,10103,girl,1,0,0.88
4,10103,girl,1,0,0.88


#### 描述统计与探索性分析

In [6]:
# groupby 可以按照某条件对数据进行分组，之后再使用 mean函数可以实现分组求均值。
data.groupby('PPED').REPEAT.mean() 

PPED
0    0.277778
1    0.191729
Name: REPEAT, dtype: float64

可以发现，没有接受过学前教育学生的留级率 (18%)高于接受过学前教育的学生 (11%)。

可视化如下：

In [7]:
data.groupby('PPED').REPEAT.mean().plot.bar()
plt.show()

<Figure size 432x288 with 1 Axes>

对比不同学校下，学前教育对留级率的影响。

其中，每一条线代表一个学校。

可见，不同学校间的学前教育对留级率的影响存在差异。但整体存在接受学前教育后留级率下降的趋势。

In [8]:
# 计算不同学校下，是否接受学前教育的留级率
data_plot = data.groupby(["SCHOOLID",'PPED']).REPEAT.mean() 
# 根据留级率进行绘图
for schoolID,data_i in data_plot.groupby("SCHOOLID"):
    data_i.plot.line(x="PPED")
plt.xlabel("pped")             # 横坐标为是否接受学前教育
plt.xticks([0,1],["no","yes"]) # 0为否，1为是
plt.ylabel(r"REPEAT \%")       # 纵坐标为留级率

Text(0, 0.5, 'REPEAT \\%')

<Figure size 432x288 with 1 Axes>

### (3) 选择模型

由于因变量(是否留级)是二分离散变量，因此我们选择使用基于伯努利(Bernoulli)分布的广义线性模型(Generalized linear model，GLM)进行模型拟合。

这里我们考虑三种可能的模型：
- 模型1：仅包含变化截距的模型。表示拟合不同学校的平均留级率。
- 模型2：变化截距模型。在模型1的基础上，考虑学前教育在level1 (group)的总体效应，这种总体效应也成为固定效应。
- 模型3：变化斜率模型。在模型2的基础上，考虑学前教育在level2 (population)的不同学校上的不同效应，这种总体效应也成为变化效应。变化截距也属于变化效应。

#### 模型1：仅包含变化截距的模型

$$
y_{(i,j)} \sim Bernoulli(p_{(i,j)})
$$
该模型只拟合不同学校的平均留级率。
- 其中$i$为不同学生，$j$为不同学校。

$$
p_{(i,j)} = g(\alpha_j) \\
\alpha_j \sim Normal(\mu_{\alpha},\sigma_{\alpha})
$$
- $\alpha_{j}$ 为截距，代表平均留级率。该解决随着不同学校j进行变化，因此称为变化截距。
- g()为链接函数
- 其中，$\mu_{\alpha}, \sigma_{\alpha}$ 为 level 1 group 的效应，而 $\alpha_{j}$ 为 level 2 population 的效应。
- 注意，此时的模型没有考虑自变量 **学前教育** 的影响。

**先验**

此时模型参数 ($\alpha$)的先验与 $\mu_{\alpha}, \sigma_{\alpha}$有关。而 $\mu_{\alpha}, \sigma_{\alpha}$ 也服从于不同的分布。比如：

$$
\mu_{\alpha} \sim Normal(0,100) \\ 
\sigma_{\alpha} \sim HalfNormal(50)
$$
- 其中 $\mu_{\alpha}, \sigma_{\alpha}$ 称为超参 (hyperparameters)，可以理解为参数 $\alpha$ 的参数。
- $Normal(0,100),HalfNormal(50)$ 称为超先验 (hyperprior)，即先验的先验。

In [9]:
# 将数据分层变量"学校(school)"转换为因子(factor)类型
school_idxs, school = pd.factorize(data.SCHOOLID)
# 定义学校与数据的映射：即标注哪名学生(行)属于哪一所学校
coords = {
    "school": school,
    "obs_id": np.arange(len(school_idxs)),
}

In [10]:
with pm.Model(coords=coords) as model1:
    # 定义超先验 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=0.0, sigma=100)  # 对应上述公式
    sigma_alpha = pm.HalfNormal("sigma_alpha", 50)

    # 定义先验
    alpha = pm.Normal('alpha', mu=mu_alpha,sd=sigma_alpha, dims="school")
    
    # 定义数据分层变量"学校(school)"
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    
    # 定义线性模型：p是确定性变化变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha[school_idx])
    
    # 定义似然函数：Y的观测值，这是一个特殊的观测变化变量，表示模型数据的可能性。
    # 也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

我们通过 PyMC3自带的函数 `model_to_graphviz()` 将模型可视化。

需要注意的是：
- 我们假设 $\alpha$ (先验)服从一个正态分布 其均值为 *mu_alpha*，误差为 *sigma_alpha*。
- 并且*mu_alpha* 和 *sigma_alpha* 各自服从不同的分布，如下图，这些分布为 **超先验(hyperpriors)**。
- 定义超先验的目的在于约束学校变量带来的差异，这被称为 "shrinkage"，是分层模型 partial pooling的效果。更多详情请参考 https://mc-stan.org/rstanarm/articles/pooling.html

In [11]:
pm.model_to_graphviz(model1)

#### 模型2：变化截距模型

在模型1的基础上，考虑学前教育在level1的总体效应。
$$
y_{(i,j)} \sim Bernoulli(p_{(i,j)}) \\
p_{(i,j)} = g(\alpha_j + \beta*\text{x}_{i, j}) \\
\alpha_j \sim Normal(\mu_{\alpha},\sigma_{\alpha})
$$
- $x$为自变量是否接受学前教育。
- 需要注意的是，学前教育的效应 $\beta$ 不随学校进行变化。这意味着它在不同学生和不同学校间保持固定，因此也称为**固定效应**。


**先验**

$$
\mu_{\alpha} \sim Normal(0,100) \\ 
\sigma_{\alpha} \sim HalfNormal(50) \\
\beta \sim Normal(0,100) \\ 
$$

In [12]:
with pm.Model(coords=coords) as model2:
    # 定义超先验 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=0.0, sigma=100)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 50)

    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="school")
    
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data.PPED, dims="obs_id")
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
   
    #####################################
    # 与模型1的区别在于定义固定效应 beta, 以及线性公式p的变化
    #####################################
    beta = pm.Normal('beta',mu=0,sd=100)
    p = pm.Deterministic("p", alpha[school_idx] + beta * x)
    
    # Y的观测值，这是一个特殊的观测变化变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

In [13]:
pm.model_to_graphviz(model2)

#### 模型3：变化截距+斜率模型

$$
y_{(i,j)} \sim Bernoulli(p_{(i,j)})
p_{(i,j)} = g(\alpha_j + \beta*\text{x}_{i, j}) \\
\alpha_j \sim Normal(\mu_{\alpha},\sigma_{\alpha}) \\
\beta_j \sim Normal(\mu_{\beta},\sigma_{\beta})
$$
在模型2的基础上，考虑学前教育在level2的不同学校上的不同效
- 此时，$x$的效应 $\beta_{j}$ 随不同的学校 j 进行变化。
- 这种总体效应也成为变化效应。其实变化截距也属于变化效应。


**先验**

$$
\mu_{\alpha} \sim Normal(0,100) \\ 
\sigma_{\alpha} \sim HalfNormal(50) \\
\mu_{\beta} \sim Normal(0,100) \\ 
\sigma_{\beta} \sim HalfNormal(50) \\
$$

In [14]:
with pm.Model(coords=coords) as model3:
    #####################################
    # 与模型2的区别在于，额外定义效应beta的超先验 Hyperpriors
    #####################################
    mu_alpha = pm.Normal("mu_alpha", mu=0.0, sigma=100)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 50)
    mu_beta = pm.Normal("mu_beta", mu=0.0, sigma=100)
    sigma_beta = pm.HalfNormal("sigma_beta", 50)
    
    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="school")
    beta = pm.Normal('beta',mu=mu_beta,sd=sigma_beta, dims="school")
    
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data.PPED, dims="obs_id")
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    
    #####################################
    # 与模型2的区别在于，注意此时的beta随着学校school_idx进行变化
    #####################################
    p = pm.Deterministic("p", alpha[school_idx] + beta[school_idx] * x)
    
    # Y的观测值，这是一个特殊的观测变化变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

注意观察，每个变化效应的先验 (alpha 和 beta)都存在超先验 (hyperpriors)。

同样，beta参数的超先验约束了不同学校间学前教育效应的影响，可避免出现极端值。

In [15]:
pm.model_to_graphviz(model3)

这里我们先考虑最后一个模型 (模型3)，在后面模型比较的部分，再考虑其他两个模型。

### (4)选择先验

In [16]:
with model3:
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=500)

首先查看 p 的先验分布。

注意，这里没有用链接函数对 p 进行转换，因此其范围在 -200到200之间，而不是 0-1之间。

In [17]:
az.plot_density(
    {'p':prior_checks['p']}
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (500) than draws (338). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 432x288 with 1 Axes>

其次，查看模型截距和斜率的先验。

In [18]:
az.plot_density(
    {'alpha':prior_checks['alpha'],
    'beta':prior_checks['beta']}
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (500) than draws (19). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 993.6x331.2 with 2 Axes>

最后，别忘了查看超先验所对应超参数(hyperparameters)的先验分布。

In [19]:
az.plot_density(
    {
        'mu_alpha':prior_checks['mu_alpha'],
        'sigma_alpha':prior_checks['sigma_alpha'],
        'mu_beta':prior_checks['mu_beta'],
        'sigma_beta':prior_checks['sigma_beta']
    }
    )
plt.show()

<Figure size 1490.4x662.4 with 6 Axes>

### (5) 拟合数据

拟合数据需要注意，虽然我们主要探究的自变量为"是否接受学前教育"，但是数据分层结构变量"学校"也是非常重要的，不要忽视了。

In [20]:
data.head()

,SCHOOLID,SEX,PPED,REPEAT,MSESC
0,10103,girl,0,0,0.88
1,10103,girl,0,0,0.88
2,10103,girl,1,0,0.88
3,10103,girl,1,0,0.88
4,10103,girl,1,0,0.88


In [21]:
data.groupby('SCHOOLID').REPEAT.mean().plot.bar()
plt.show()

<Figure size 432x288 with 1 Axes>

In [22]:
# 将数据分层变量"学校(school)"转换为因子(factor)类型
school_idxs, school = pd.factorize(data.SCHOOLID)
# 定义学校与数据的映射：即标注哪名学生(行)属于哪一所学校
coords = {
    "school": school,
    "obs_id": np.arange(len(school_idxs)),
}

with pm.Model(coords=coords) as model3:
    # 定义level2学校的 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=0.0, sigma=100)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 50)
    mu_beta = pm.Normal("mu_beta", mu=0.0, sigma=100)
    sigma_beta = pm.HalfNormal("sigma_beta", 50)
    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="school")
    beta = pm.Normal('beta',mu=mu_beta,sd=sigma_beta, dims="school")
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data.PPED, dims="obs_id")
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    # 定义线性模型：p是确定性变化变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha[school_idx] + beta[school_idx] * x)
    # Y的观测值，这是一个特殊的观测变化变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

### (6)采样过程诊断

如果使用MCMC对后验进行近似，则需要首先对MCMC过程进行评估。

* 是否收敛；
* 是否接近真实的后验。

对采样过程的评估我们会采用目视检查或rhat这个指标

In [23]:
with model3:
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace3 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma_beta, mu_beta, sigma_alpha, mu_alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 24 seconds.


0, dim: obs_id, 338 =? 338


There were 29 divergences after tuning. Increase `target_accept` or reparameterize.
There were 339 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.752539189964468, but should be close to 0.9. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


In [24]:
az.plot_trace(trace3, var_names=['alpha','beta'])
plt.show()

<Figure size 864x288 with 4 Axes>

注意，由于参数会随着学校进行变化，因此每一所学校(19所学校)都对应一个参数，如下：

In [25]:
az.summary(trace3, var_names=['alpha','beta'], kind="diagnostics")

,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],0.029,0.021,323.0,1227.0,1.01
alpha[1],0.014,0.010,850.0,1607.0,1.01
alpha[2],0.019,0.013,611.0,1257.0,1.02
alpha[3],0.025,0.018,447.0,1065.0,1.02
alpha[4],0.035,0.025,251.0,1148.0,1.01
alpha[5],0.010,0.007,1211.0,1739.0,1.01
alpha[6],0.013,0.009,864.0,1423.0,1.02
alpha[7],0.035,0.025,261.0,1012.0,1.01
alpha[8],0.015,0.011,660.0,321.0,1.02
alpha[9],0.017,0.013,731.0,1455.0,1.01


### (7)模型诊断

在MCMC有效的前提下，需要继续检验模型是否能够较好地拟合数据。

我们会使用后验预测分布通过我们得到的参数生成一批模拟数据，并将其与真实数据进行对比。

In [26]:
# 后验预测分布的计算仍在容器中进行
with model3:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace3.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace3, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [27]:
# 绘制后验预测分布
az.plot_ppc(trace3)
plt.show()

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


<Figure size 432x288 with 1 Axes>

### (8)模型比较

前面的诊断过程，我们只考虑了模型3 (变化截距+斜率模型)。在模型比较阶段，我们可以同时比较三个模型：
- 模型1：仅包含变化截距的模型，拟合不同学校的平均留级率。
- 模型2：变化截距模型。在模型1的基础上，考虑学前教育在group level的总体效应。
- 模型3：变化截距+斜率模型。在模型2的基础上，考虑学前教育在不同学校上 (population level)的不同效应。

In [28]:
# 对模型进行mcmc采样
with model1:
    trace1 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)
with model2:
    trace2 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)
# 模型3前面完成了采样，这里可以不用再采样了
# with model3:
#     trace2 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [alpha, sigma_alpha, mu_alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 10 seconds.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


0, dim: obs_id, 338 =? 338


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma_alpha, mu_alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 12 seconds.
There were 49 divergences after tuning. Increase `target_accept` or reparameterize.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


0, dim: obs_id, 338 =? 338


In [29]:
# 将三个模型的采样结果进行比较
compare_dict = {
    "仅包含截距的模型": trace1, 
    "变化截距模型": trace2, 
    "变化斜率模型": trace3
    }
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='loo')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
变化斜率模型,0,-171.263208,12.461399,0.000000,6.657864e-01,10.313712,0.000000,False,log
仅包含截距的模型,1,-171.747158,10.166437,0.483949,3.342136e-01,10.156738,1.729660,False,log
变化截距模型,2,-171.944453,10.723415,0.681245,4.440892e-16,10.213282,1.211617,False,log


结果显示， 变化截距+斜率模型的拟合度好于其他两个模型，这表示，在不同学校间学前教育对留级率的影响是不同的。

### (9)统计推断

通过模型比较可以发现，学校所带来的变化效应的影响。如果体现这种影响呐？

通过 arviz提供的函数 `plot_forest()` 可以可视化在不同学校间学前教育所带来的影响。
- 图中左边的编号是 学校ID。
- 94% HDI 是学前教育效应 $\beta$ 在不同学校间的可信区间。
- 通过 94% HDI判断效应是否可信发现，学前教育效应在一些学校间 (比如10418, 20204)存在差异，而在一些学校 (比如10109， 20309)不存在差异。

In [30]:
sort_ = trace3.posterior["beta"].mean(dim=["chain", "draw"])
sort_ = sort_.sortby(sort_, ascending=True)

az.plot_forest(
    trace3,
    var_names=["beta"],
    combined=True,
    coords={"school": sort_["school"]}
)
plt.show()

<Figure size 432x424.8 with 1 Axes>

同样，我们可以通过 `az.summary` 函数查看各参数的具体数值。这里只展示了 beta参数的数值，大家可自行查看 alpha参数的相关统计值。

In [31]:
az.summary(trace3, var_names = ["beta"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],-0.947,0.713,-2.269,0.263,0.048,0.034,199.0,932.0,1.01
beta[1],-0.176,0.577,-1.257,0.968,0.020,0.015,763.0,1690.0,1.01
beta[2],-0.618,0.558,-1.635,0.535,0.019,0.013,842.0,1817.0,1.02
beta[3],0.085,0.599,-1.102,1.170,0.028,0.020,458.0,1097.0,1.01
beta[4],-1.290,0.835,-2.753,0.138,0.080,0.057,124.0,1198.0,1.01
beta[5],-0.680,0.906,-2.262,1.018,0.094,0.067,100.0,271.0,1.03
beta[6],-0.846,0.667,-2.154,0.273,0.040,0.028,284.0,1002.0,1.01
beta[7],-1.079,0.694,-2.310,0.311,0.034,0.024,393.0,1226.0,1.01
beta[8],-0.386,0.701,-1.697,0.982,0.034,0.024,336.0,879.0,1.01
beta[9],-0.888,0.696,-2.199,0.370,0.053,0.038,178.0,1251.0,1.01


前面的分析是针对 population level 参数，即变化效应。而我们一般想了解组层面 (group level)的效应是否存在，即固定效应。

组层面的效应对应的参数是 mu_alpha, mu_beta, sigma_alpha, sigma_beta。我们可以通过 `az.summary` 与 `az.plot_posterior` 来查看这些参数。

In [32]:
az.plot_posterior(trace3, var_names=['mu_alpha','mu_beta'])
plt.show()

<Figure size 993.6x331.2 with 2 Axes>

In [33]:
az.summary(trace3, var_names=['mu_alpha','mu_beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_alpha,-1.014,0.303,-1.613,-0.454,0.014,0.010,429.0,871.0,1.0
mu_beta,-0.583,0.394,-1.290,0.196,0.021,0.015,342.0,899.0,1.0


whoops！ 别忘了广义线性模型中链接函数的存在，我们需要把上述的参数值转换到0-1的范围。

可以通过 inv.logit函数进行转换，即 $1 / (1 + exp(-\theta))$。

In [34]:
alpha = 1 / (1 + np.exp(-trace3.posterior["mu_alpha"].mean())).to_pandas()
beta = 1 / (1 + np.exp(-(trace3.posterior["mu_beta"].mean()))).to_pandas()
print(
    "未接受学前教育学生的留级率(%) = ",alpha, 
    "\n 接受学前教育学生的留级率(%) = ", alpha + beta,
    "\n 学前教育对留级率的影响(%) = ", beta
    )

未接受学前教育学生的留级率(%) =  0.26611387752965954 
 接受学前教育学生的留级率(%) =  0.6244331002322159 
 学前教育对留级率的影响(%) =  0.3583192227025564


最后，我们比较，模型3(变化截距+斜率)和模型2(变化截距模型)中学前教育组层面的效应是否相同。

可见，两个模型的估计非常类似。但是变化截距+斜率模型比变化截距模型更多地考虑了学前教育效应在不同学校间的差异。

In [35]:
beta1 = 1 / (1 + np.exp(-(trace2.posterior["beta"].mean()))).to_pandas()
beta2 = 1 / (1 + np.exp(-(trace3.posterior["mu_beta"].mean()))).to_pandas()
print(
    "变化截距模型中学前教育的效应(%) = ", beta1, 
    "\n 变化截距+斜率模型中学前教育的效应(%) = ", beta2
    )

变化截距模型中学前教育的效应(%) =  0.3881406175872315 
 变化截距+斜率模型中学前教育的效应(%) =  0.3583192227025564


### 练习

前面我们定义了各种分层模型，但我们没有演示只包含变化斜率的模型，如下图。

接下来的练习大家可以尝试以下定义只包含变化斜率的贝叶斯广义线性分层模型。

关键在于：
- 只允许斜率 beta 随学习变化。而截距 alpha 不变化。
- 如何定义分层模型的先验以及相应的超先验。

![Image Name](https://cdn.kesci.com/upload/image/rmank74ytm.png?imageView2/0/w/640/h/640)

结合上节课学习的GLM和HLM，若我们的数据因变量为离散数据，且数据为嵌套结构时，我们可以将HLM和GLM结合。



In [36]:
#加载需要使用的库
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm

np.random.seed(123)  # 变化数种子，确保随后生成的变化数相同

# 加载数据
data = pd.read_csv("/home/mw/input/data1317/thaieduc.csv")
data.PPED = data.PPED.map({'yes':1,'no':0})

# 将数据分层变量"学校(school)"转换为因子(factor)类型
school_idxs, school = pd.factorize(data.SCHOOLID)
# 定义学校与数据的映射：即标注哪名学生(行)属于哪一所学校
coords = {
    "school": school,
    "obs_id": np.arange(len(school_idxs)),
}

In [37]:
with pm.Model(coords=coords) as model_practice:
    ############################################################################
    # 要求：定义level2学校的 Hyperpriors
    # 提示：通过 mu_beta = pm.Normal(???) 进行定义
    ############################################################################
    mu_beta = pm.Normal("mu_beta", ...) 
    sigma_beta = pm.HalfNormal("sigma_beta", ...)  # 注意这是 HalfNormal 分布
    
    ############################################################################
    # 要求：定义先验
    # 提示：通过 beta = pm.Normal(???) 进行定义。通过 dims="school" 使得 beta随学校变化
    ############################################################################
    beta = pm.Normal('beta', ...)
    alpha = pm.Normal('alpha', ...)

    # 定义数据分层变量"学校(school)"
    school_idx = pm.Data("school_idx", school_idxs, dims="obs_id")
    x = pm.Data("x", data.PPED, dims="obs_id")
    # 定义线性模型：p是确定性变化变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha + beta[school_idx]*x)
    # 定义似然函数：Y的观测值，这是一个特殊的观测变化变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data.REPEAT, dims="obs_id")

TypeError: float() argument must be a string or a number, not 'ellipsis'

In [ ]:
pm.model_to_graphviz(model_practice)

In [ ]:
with model_practice:
    trace_practice = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

In [ ]:
az.summary(trace_practice, var_names=['mu_beta'])

In [ ]:
az.plot_posterior(trace_practice, var_names=['mu_beta'])
plt.show()

## Part 3: 示例2——信号检测论

### Workflow

![Image Name](https://cdn.kesci.com/upload/image/rkvikqg9q6.png?imageView2/0/w/650/h/650)

### (1) 提出研究问题

假如我们在进行一项再认实验，这个实验向被试呈现了一组图片，这些图片中有些是新的（被试之前没有见过的），而有些是旧的（被试之前已看过的）。

被试需要对呈现的每一张图片进行反应：“新（未见过）”或者“旧（见过）”。

信号检测论(Signal Detection Theory, SDT)通过分别测量被试的敏感性以及判断标准来建立每个被试再认过程的模型。

信号检测论模型的基本理念是：在每个试次中，当刺激呈现时，被试心里会产生一种“熟悉”（或者记忆强度）的信号。随后被试会依据这些信号，判断当前的刺激是“新”或“旧”。
![Image Name](https://th.bing.com/th/id/R.1565c09f95a7411070316c78242814c8?rik=rKBAJumpMrZ5Qg&riu=http%3a%2f%2fmedia.open.com.cn%2fmedia_file004%2f1003%2fdongshi%2fshiyanxlx%2fchapter6%2fimage%2f01.jpg&ehk=ui4XLQbTaXgMxP3BuNoBRAnww2cCrMLys2tN%2f1vm%2fvA%3d&risl=&pid=ImgRaw&r=0)

被试的反应有信号检测论的参数d'（辨别力指标/感觉敏感性指标）和 c（报告标准）决定。


那么我们该如何计算被试的参数d'（辨别力指标/感觉敏感性指标）和 c（报告标准）？

### (2) 数据收集

我们使用Skagerberg和Wright记忆一致性(memory conformity)研究中的对照组(Skagerberg & Wright，2008)

In [3]:
# 加载数据
data = pd.read_csv("/home/mw/input/data1317/confcontr.csv")
# 查看数据
data.head()
# 该数据中每一行都是一个试次，subno为被试编号，sayold为被试的新旧判断，isold为图片本身的新旧程度

#### 描述统计与探索性分析

首先，我们可以查看单个被试每个试次的反应（击中率(hit)、虚报率(false alarm)、正确拒斥率(correct rejection)、漏报率(miss)）

In [55]:
data = data.loc[data['subno']==53]
hr = len(data.loc[(data['sayold']==1)&(data['isold']==1)])/len(data.loc[data['isold']==1])
mr = len(data.loc[(data['sayold']==0)&(data['isold']==1)])/len(data.loc[data['isold']==1])
crr = len(data.loc[(data['sayold']==0)&(data['isold']==0)])/len(data.loc[data['isold']==0])
far = len(data.loc[(data['sayold']==1)&(data['isold']==0)])/len(data.loc[data['isold']==0])
print(f"hr is {hr} ")
print(f"mr is {mr} ")
print(f"crr is {crr} ")
print(f"far is {far} ")

### (3) 选择模型

最常见的SDT模型，即假定被试对两种信号的熟悉程度的分布属于高斯分布（即正态分布），且这两个分布的方差是齐性的，但均值可能不同（例如，对旧刺激的熟悉程度更高）。

在方差齐性的情况下，对于单个被试，d'可由标准化后的击中率和虚报率(hit and false alarm rates)之差计算得来(Stanislaw & Todorov，1999)

$d'=\phi^{-1}(HR)-\phi^{-1}(FAR)$

$\phi$是累积正态密度函数(cumulative normal density function)，可将z分数转换为概率，其反函数$\phi^{-1}$ 将概率（例如击中率或虚报率）转换为z分数。标准化后的击中率和虚报率分别表示为 $z(HR)$ 和$z(FAR)$。

反应标准$c=-z(FAR)$（负虚报率的标准分数）表示(DeCarlo，1998)。



在除了直接对d'和c进行计算，我们可以使用广义线性模型来进行计算。

在二项反应的GLM中，我们假定反应属于Bernoulli分布.

即：$y_i \sim Bernoulli(p_i)$

被试反应即对图片新旧程度的判断是由击中率(HR)或虚报率(FAR)来决定的，故概率p表示击中率(HR)。

概率p本身的范围为0-1，因此，我们不能直接用线性模型直接对p进行建模。

相反，我们用一个连结函数将p转化为‘线性预测值’ $η$，并用线性回归模型对$η$建模。

即：$p_i = \phi(\eta_i)$

如前所述，$\phi$是一个累积正态密度函数并能将z 值转化为概率，那我们可以用一个截距和斜率来表示$\eta$。


之所以用$\phi$而不是logit函数，是因为击中率或虚报率本身是一个概率，它是由信号分布的z分数转化而来，而不是简单地将至于范围转化到[0,1]

即：$\eta_i = \beta_0 + \beta_1 * isold_i$

由于$d'=\phi^{-1}(HR)-\phi^{-1}(FAR) = z(HR) - z(FAR)$，$c=-z(FAR)$

当isold为0，反应由虚报率决定，即$z(FAR) = -c$

当isold为1，反应由击中率决定，即$z(HR) = z(HR) - z(FAR) + z(FAR) = -c+d$

故在线性模型中，$\beta_0$代表c(报告标准)的相反数，$\beta_1$代表d(辨别力指标)


我们首先展示一个单被试模型的例子，由于该模型只有一个被试，故是一个单层的模型
$$
y_{(i)} \sim Bernoulli(p_{(i)}) \\
p_{(i)} = \phi(\eta) \\
\eta = -c + d *x_{isold}
$$


**先验**

$$
\ c \sim Normal(0,1) \\
\ d \sim Normal(0,1) \\ 
$$

In [59]:
with pm.Model(coords=coords) as model0:
    # 定义priors
    c = pm.Normal("c", mu=0.0, sigma=1)  
    d = pm.Normal("d", mu=0.0, sigma=1)
    # 自变量x是之前已经载入的数据
    x = pm.Data("x", data['isold'])
    # 定义线性模型：p是确定性变化变量，这个变量的值完全由右端值确定
    eta = pm.Deterministic("eta", -c + d*x)
    # 定义似然函数：Y的观测值，这是一个特殊的观测变化变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    sayold = pm.Bernoulli("syaold", logit_p=eta, observed=data['sayold'])

In [60]:
pm.model_to_graphviz(model0)

单被试模型仅为简单说明如何用GLM建立信号检测论，接下来的过程为了避免混淆将不对该模型做进一步的分析

接下来我们将展示在分层结构中，我们如何利用GLMM建立信号检测论的模型

In [74]:
data = pd.read_csv("/home/mw/input/data1317/confcontr.csv")
data.head()

#### 模型1：变化截距模型

我们假定各被试的c(报告标准)是不同的

**固定效应**
$$
y_{i,j} \sim Bernoulli(p_{i,j}) \\
p_{i,j} = \phi(\eta_{i,j}) \\
\eta_{i,j} = -c_{i,j} + d_{i,j} *x_{isold,i,j}
$$


**先验**

$$
\ c_{i,j} \sim Normal(\mu_{c,j},\sigma_{c,j}) \\
\ d_{i,j} \sim Normal(0,1) \\ 
$$
**随机效应**
$$
\ \mu_{c,j} \sim Normal(0,1) \\
\ \sigma_{c,j} \sim HalfNormal(1) \\
$$

In [87]:
# 将数据分层变量"被试(subject)"转换为因子(factor)类型
subj_idxs, subject = pd.factorize(data.subno)
# 定义学校与数据的映射：即标注哪名学生(行)属于哪一所学校
coords = {
    "subject": subject,
    "obs_id": np.arange(len(subj_idxs)),
}

In [88]:
with pm.Model(coords=coords) as model1:
    # 定义level2被试的 Hyperpriorspriors
    mu_c = pm.Normal("mu_c", mu=0.0, sigma=1)  
    sigma_c = pm.HalfNormal("sigma_c", sigma = 1)
    # 定义priors
    c = pm.Normal("c", mu=mu_c, sigma=sigma_c,dims="subject")  
    d = pm.Normal("d", mu=0.0, sigma=1)
    # 定义数据分层变量"被试(subject)"
    subj_idx = pm.Data("subject", subj_idxs, dims="obs_id")
    # 自变量x是之前已经载入的数据
    x = pm.Data("x", data['isold'], dims="obs_id")
    # 定义线性模型：p是确定性变化变量，这个变量的值完全由右端值确定
    eta = pm.Deterministic("eta", -c[subj_idx] + d*x)
    # 定义似然函数：Y的观测值，这是一个特殊的观测变化变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    sayold = pm.Bernoulli("syaold", logit_p=eta, observed=data['sayold'],dims="obs_id")

In [89]:
pm.model_to_graphviz(model1)

#### 模型2：变化斜率模型

我们假定各被试的d(辨别力指标)是不同的

**固定效应**
$$
y_{i,j} \sim Bernoulli(p_{i,j}) \\
p_{i,j} = \phi(\eta_{i,j}) \\
\eta_{i,j} = -c_{i,j} + d_{i,j} *x_{isold,i,j}
$$


**先验**

$$
\ c_{i,j} \sim Normal(0,1) \\
\ d_{i,j} \sim Normal(\mu_{d,j},\sigma_{d,j}) \\ 
$$
**随机效应**
$$
\ \mu_{d,j} \sim Normal(0,1) \\
\ \sigma_{d,j} \sim HalfNormal(1) \\
$$

In [92]:
with pm.Model(coords=coords) as model2:
    # 定义level2被试的 Hyperpriorspriors
    mu_d = pm.Normal("mu_d", mu=0.0, sigma=1)  
    sigma_d = pm.HalfNormal("sigma_d", sigma = 1)
    # 定义priors
    c = pm.Normal("c", mu=0, sigma=1)  
    d = pm.Normal("d", mu=mu_d, sigma=sigma_d, dims="subject")
    # 定义数据分层变量"被试(subject)"
    subj_idx = pm.Data("subject", subj_idxs, dims="obs_id")
    # 自变量x是之前已经载入的数据
    x = pm.Data("x", data['isold'], dims="obs_id")
    # 定义线性模型：p是确定性变化变量，这个变量的值完全由右端值确定
    eta = pm.Deterministic("eta", -c + d[subj_idx]*x)
    # 定义似然函数：Y的观测值，这是一个特殊的观测变化变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    sayold = pm.Bernoulli("syaold", logit_p=eta, observed=data['sayold'],dims="obs_id")

In [93]:
pm.model_to_graphviz(model2)

### (4)选择先验

In [103]:
with model1:
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=500)

In [104]:
az.plot_density(
    {'eta':prior_checks['eta']}
    )
plt.show()

In [108]:
with model2:
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=500)

In [109]:
az.plot_density(
    {'eta':prior_checks['eta']}
    )
plt.show()

### (5) 拟合数据

In [110]:
with model1:
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace1 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

In [111]:
with model2:
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace2 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

### (6)采样过程诊断

如果使用MCMC对后验进行近似，则需要首先对MCMC过程进行评估。

* 是否收敛；
* 是否接近真实的后验。

对采样过程的评估我们会采用目视检查或rhat这个指标

In [112]:
az.summary(trace1, var_names = ["mu_c","sigma_c","d"])

In [115]:
az.summary(trace2, var_names = ["mu_d","sigma_d","c"])

In [116]:
az.plot_trace(trace1, var_names = ["mu_c","sigma_c","d"])
plt.show()

In [117]:
az.plot_trace(trace2, var_names = ["mu_d","sigma_d","c"])
plt.show()

### (7)模型诊断

在MCMC有效的前提下，需要继续检验模型是否能够较好地拟合数据。

我们会使用后验预测分布通过我们得到的参数生成一批模拟数据，并将其与真实数据进行对比。

In [118]:
# 后验预测分布的计算仍在容器中进行
with model1:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace1.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace1, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

In [120]:
# 后验预测分布的计算仍在容器中进行
with model2:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace2.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace2, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

In [121]:
# 绘制后验预测分布
az.plot_ppc(trace1)
plt.show()

In [ ]:
# 绘制后验预测分布
az.plot_ppc(trace2)
plt.show()

### (8)模型比较

在模型比较阶段，我们可以同时比较2个模型：
- 模型1：变化截距模型。
- 模型2：变化截距+斜率模型。

In [122]:
# 将2个模型的采样结果进行比较
compare_dict = {
    "变化截距模型": trace1, 
    "变化截距+斜率模型": trace2
    }
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='loo')
comp

In [123]:
# 将2个模型的采样结果进行比较
compare_dict = {
    "变化截距模型": trace1, 
    "变化截距+斜率模型": trace2
    }
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='waic')
comp

### 总结

- 本节课学习了分层线性模型的基本概念：包括变化截距与变化斜率、固定效应与变化效应、group level与population level。
- 通过 PyMc 分别拟合变化截距模型与变化斜率模型(包含变化截距)。